In [1]:
import configparser
import psycopg2
import time
from sql_queries import copy_table_queries, insert_table_queries
from create_tables import create_tables, drop_tables

In [2]:
def load_staging_tables(cur, conn):
    for query in copy_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()

def insert_tables(cur, conn):
    for query in insert_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()

In [3]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

print("1. Dropping tables...")
start_time = time.time()
drop_tables(cur, conn)
print("--- It took %s seconds ---" % (time.time() - start_time))

print("2. Creating tables...")
start_time = time.time()
create_tables(cur, conn)
print("--- It took %s seconds ---" % (time.time() - start_time))

#print("3. Loading staging tables, it may take time...")
#start_time = time.time()
#load_staging_tables(cur, conn)
#print("--- It took %s seconds ---" % (time.time() - start_time))

print("4. Loading final tables...")
start_time = time.time()
insert_tables(cur, conn)
print("--- It took %s seconds ---" % (time.time() - start_time))

conn.close()

1. Dropping tables...
--- It took 3.188786506652832 seconds ---
2. Creating tables...
--- It took 4.978750228881836 seconds ---
4. Loading final tables...

        INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
        SELECT 
            TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second' AS start_time,
            userid AS user_id,
            level,
            s.song_id AS song_id,
            a.artist_id AS artist_id,
            sessionid AS sessions_id,
            ste.location,
            useragent AS user_agent
            FROM staging_events AS ste
                JOIN songs AS s ON ste.song = s.title
                JOIN artists AS a ON ste.artist = a.name
 

        INSERT INTO users (user_id, first_name, last_name, gender, level)
        SELECT 
            DISTINCT(userid) as user_id,
            firstname as first_name,
            lastname as last_name,
            gender,
            level
        FROM staging_ev